In [1]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cpu


In [3]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [4]:
#Dataloader

# #Path for training and testing directory
# drive.mount('/content/drive')
# dataset_path = "/content/drive/My Drive/Colab Notebooks/Agrar_uchun/v"
# dataset = torchvision.datasets.ImageFolder(root = dataset_path, transform=transformer)

dataset_path_local = r'C:\Users\mirja\myenv\DjangoAPI\selxoz_project\edited_image'
dataset = torchvision.datasets.ImageFolder(root = dataset_path_local, transform=transformer)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# train_path=dataset_path
# test_path=dataset_path

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [5]:
# #categories
# root = pathlib.Path(dataset_path)

root = pathlib.Path(dataset_path_local)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [6]:
print(classes)

['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']


In [7]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet,self).__init__()

        #Output size after convolution filter
        #((w-f+2P)/s) +1

        #Input shape= (256,3,256, 256)

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,256, 256)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,256, 256)
        self.relu1=nn.ReLU()
        #Shape= (256,12,256, 256)

        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,128,128)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,128,128)
        self.relu2=nn.ReLU()
        #Shape= (256,20,128,128)



        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,128,128)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,128,128)
        self.relu3=nn.ReLU()
        #Shape= (256,32,128,128)


        self.fc=nn.Linear(in_features=128 * 128 * 32,out_features=num_classes)



        #Feed forwad function

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.relu2(output)

        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)


            #Above output will be in matrix form, with shape (256,32,128, 128)

        output=output.view(-1,32*128*128)


        output=self.fc(output)

        return output




In [8]:
model=ConvNet(num_classes=10).to(device)

In [9]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [10]:
num_epochs=10

In [11]:
#calculating the size of training and testing images
# train_count=len(glob.glob(train_path+'/**/*.JPG'))
# test_count=len(glob.glob(test_path+'/**/*.jpg'))
train_count=len(train_dataset)
test_count=len(test_dataset)


In [12]:
print(train_count,test_count)

800 200


In [13]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        optimizer.zero_grad()

        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()


        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count


    # Evaluation on testing dataset
    model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy




Epoch: 0 Train Loss: tensor(125.5011) Train Accuracy: 0.16625 Test Accuracy: 0.11
Epoch: 1 Train Loss: tensor(33.9502) Train Accuracy: 0.38875 Test Accuracy: 0.23
Epoch: 2 Train Loss: tensor(13.8202) Train Accuracy: 0.60375 Test Accuracy: 0.25
Epoch: 3 Train Loss: tensor(6.6492) Train Accuracy: 0.75625 Test Accuracy: 0.52
Epoch: 4 Train Loss: tensor(3.9384) Train Accuracy: 0.8225 Test Accuracy: 0.48
Epoch: 5 Train Loss: tensor(2.7900) Train Accuracy: 0.86375 Test Accuracy: 0.665
Epoch: 6 Train Loss: tensor(1.3508) Train Accuracy: 0.9225 Test Accuracy: 0.58
Epoch: 7 Train Loss: tensor(0.6270) Train Accuracy: 0.95375 Test Accuracy: 0.56
Epoch: 8 Train Loss: tensor(0.4157) Train Accuracy: 0.965 Test Accuracy: 0.57
Epoch: 9 Train Loss: tensor(0.5458) Train Accuracy: 0.96625 Test Accuracy: 0.615


In [14]:
print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

Epoch: 9 Train Loss: tensor(0.5458) Train Accuracy: 0.96625 Test Accuracy: 0.615
